<div style="border:solid orange 3px; padding: 16px">  
    <font size="4">  
        <p style="text-align: center;">
            <b> 13. Выпускной проект. Анализ эффективности рекламных кампаний с использованием SQL </b>
        </p> 
    </font>
</div>

<a id="contents"></a>
### Оглавление:
0. [Описание проекта](#0)
1. [Подключение к БД и предобработка данных](#1)
2. [Затраты на каждый рекламный источник](#2)
3. [Выручка от самого дорогого источника рекламы](#3)
4. [Дневные затраты на пользователя по источнику](#4)

 <a id="0"></a>
### 0. Описание проекта

**Аналитика в онлайн-маркетплейсе «Заберу».**

«Заберу» проводит рекламные акции длительностью 1 день. Руководитель отдела маркетинга хочет узнать, насколько эффективны такие кампании.

Данные - информация о посещениях и заказах на сайте, а также затраты на рекламу.

 <a id="1"></a>
### 1. Подключение к БД и предобработка данных

In [ ]:
pip install psycopg2-binary

In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import re

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd' : 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db'  : 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
                                                    
engine = create_engine(connection_string, connect_args={'sslmode':'require'}, echo=False)

---

In [3]:
# выведем содержимое таблицы visits
visits_query = ''' SELECT * from visits 
               '''

In [4]:
visits = pd.io.sql.read_sql(visits_query, con = engine)

In [5]:
visits.head()

,id,uid,device,endts,sourceid,startts
0,0,16879256277535980062,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00
1,1,104060357244891740,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00
2,2,7459035603376831527,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00
3,3,16174680259334210214,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00
4,4,9969694820036681168,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00


Таблица `visits` (лог сервера с информацией о посещениях сайта):
- `uid` — уникальный идентификатор пользователя
- `device` — категория устройства пользователя
- `startts` — дата и время начала сессии
- `endts` — дата и время окончания сессии
- `sourceid` — идентификатор рекламного источника, из которого пришёл пользователь

In [6]:
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358532 entries, 0 to 358531
Data columns (total 6 columns):
id          358532 non-null int64
uid         358532 non-null object
device      358532 non-null object
endts       358532 non-null datetime64[ns]
sourceid    358532 non-null int64
startts     358532 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 16.4+ MB


Пропусков в данных нет.

---

In [7]:
# выведем содержимое таблицы orders
orders_query = ''' SELECT * from orders 
               '''

In [8]:
orders = pd.io.sql.read_sql(orders_query, con = engine)

In [9]:
orders.head()

,id,buyts,revenue,uid
0,1,2017-06-01 00:10:00,$17.00,10329302124590727494
1,2,2017-06-01 00:25:00,$0.55,11627257723692907447
2,3,2017-06-01 00:27:00,$0.37,17903680561304213844
3,4,2017-06-01 00:29:00,$0.55,16109239769442553005
4,5,2017-06-01 07:58:00,$0.37,14200605875248379450


Таблица `orders` (информация о заказах):
- `uid` — уникальный идентификатор пользователя, который сделал заказ
- `buyts` — дата и время заказа
- `revenue` — выручка с этого заказа

In [10]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 4 columns):
id         50415 non-null int64
buyts      50415 non-null datetime64[ns]
revenue    50415 non-null object
uid        50415 non-null object
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1.5+ MB


Пропусков в данных нет.

---

In [11]:
# выведем содержимое таблицы advertisment_costs
adv_costs_query = ''' SELECT * from advertisment_costs 
                  '''

In [12]:
adv_costs = pd.io.sql.read_sql(adv_costs_query, con = engine)

In [13]:
adv_costs.head()

,id,sourceid,dt,costs
0,1,1,2017-06-01,$75.20
1,2,1,2017-06-02,$62.25
2,3,1,2017-06-03,$36.53
3,4,1,2017-06-04,$55.00
4,5,1,2017-06-05,$57.08


Таблица `advertisment_costs` (информация о затратах на маркетинг):
- `sourceid` — идентификатор рекламного источника
- `dt` — дата
- `costs` — затраты на этот рекламный источник в этот день

In [14]:
adv_costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 4 columns):
id          2542 non-null int64
sourceid    2542 non-null int64
dt          2542 non-null datetime64[ns]
costs       2542 non-null object
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 79.6+ KB


Пропусков в данных нет.

---

 <a id="2"></a>
### 2. Затраты на каждый рекламный источник  [*(Оглавление)*](#contents)

In [15]:
# сгруппируем по рекламному источнику и найдем сумму расходов
costs_per_source_query = ''' SELECT sum(costs::numeric) AS cost,   
                                    sourceid AS source_id
                             FROM advertisment_costs
                             GROUP BY sourceid
                             ORDER BY sourceid
                         '''

In [16]:
costs_per_source = pd.io.sql.read_sql(costs_per_source_query, con = engine)

In [17]:
costs_per_source

,cost,source_id
0,20833.27,1
1,42806.04,2
2,141321.63,3
3,61073.60,4
4,51757.10,5
5,5517.49,9
6,5822.49,10


---

Проверим в pandas.

In [18]:
# для каждой записи в столбце 'costs' удалим символ "$", заменим разделитель и приведем к нужному типу
adv_costs['costs'] = adv_costs.apply(lambda x: re.sub('[$]', '', x['costs']), axis = 1)\
                              .str.replace(',', '')\
                              .astype(float)

In [19]:
# сгруппируем по источнику рекламы и найдём сумму
adv_costs.groupby('sourceid')['costs'].sum()

sourceid
1      20833.27
2      42806.04
3     141321.63
4      61073.60
5      51757.10
9       5517.49
10      5822.49
Name: costs, dtype: float64

Самый дорогой рекламный источник - id 3.

 <a id="3"></a>
### 3. Выручка от самого дорогого источника рекламы  [*(Оглавление)*](#contents)
Необходимо вычислить суммарную выручку, максимальную и минимальную выручку от заказов, сделанных пользователями, которые хотя бы раз перешли на сайт с настольных ПК (тип устройства `desktop`) из рекламного источника, на который было потрачено больше всего денег.

In [20]:
desktop_most_expensive_source_revenue_query = \
''' SELECT COUNT(DISTINCT o.uid) AS unique_id,                                        -- количество уникальных uid
           COUNT(o.revenue) AS orders,                                                -- общее количество заказов
           MIN(o.revenue::numeric) AS min_order,                                      -- наименьшая стоимость заказа
           MAX(o.revenue::numeric) AS max_order,                                      -- наибольшая стоимость заказа
           SUM(o.revenue::numeric) AS sum_order                                       -- общая стоимость заказов       
                                           
    FROM orders o                                                                     -- объединим таблицы заказов и посещений 
    LEFT JOIN visits v ON o.uid = v.uid                                               -- по условию равенства uid
            
    WHERE o.buyts::TIMESTAMP BETWEEN v.startts::TIMESTAMP AND v.endts::TIMESTAMP AND  -- отфильтруем по времени
          v.uid IN (SELECT uid                                                        -- оставим только тех пользователей,
                    FROM visits                                                       -- которые посещали сайт
                    WHERE sourceid = 3 AND                                            -- с рекламного источника 3 и 
                          device = 'desktop')                                         -- с персонального компьютера 
'''

In [21]:
desktop_most_expensive_source_revenue = pd.io.sql.read_sql(desktop_most_expensive_source_revenue_query, con = engine)

In [22]:
desktop_most_expensive_source_revenue

,unique_id,orders,min_order,max_order,sum_order
0,9406,12631,0.0,1195.64,64652.22


---

Проверим в pandas.

In [23]:
visits = visits[['startts', 'endts', 'device', 'sourceid', 'uid']]

In [24]:
# соединим датасеты по условию равенства uid так, чтобы в результирующем
# наборе данных были все строки из таблицы с заказами
orders_visits = pd.merge(orders, visits, on = 'uid', how = 'left')

In [25]:
# оставим только те заказы, пользователи которых хотя бы раз заходили на сайт с рекламного источника 3 и с компьютера
orders_visits = orders_visits.loc[
                orders_visits.loc[:,'uid'].isin(orders_visits.query("(sourceid == 3) and (device == 'desktop')")['uid'])
                ]

In [26]:
# ограничим по времени (время совершения заказа должно быть между началом и окончанием посещения)
orders_visits = orders_visits.query('startts <= buyts <= endts')

In [27]:
# количество уникальных пользователей
orders_visits['uid'].nunique()

9406

In [28]:
# количество заказов
orders_visits['uid'].count()

12631

In [29]:
# для каждой записи в столбце 'revenue' удалим символ "$", заменим разделитель и приведем к нужному типу
orders_visits['revenue'] = orders_visits.apply(lambda x: re.sub('[$]', '', x['revenue']), axis = 1)\
                              .str.replace(',', '')\
                              .astype(float)

In [30]:
# наименьшая выручка
orders_visits['revenue'].min()

0.0

In [31]:
# наибольшая выручка
orders_visits['revenue'].max()

1195.64

In [32]:
# суммарная выручка
orders_visits['revenue'].sum().round(2)

64652.22

- Количество пользователей, которые хотя бы раз переходили на сайт с компьютера от самого дорого рекламного источника - `9406`. 
- Эти пользователи совершили `12631` заказов.
- Самая маленькая выручка за заказ - `0$`; самая большая `1195$`.
- Суммарная выручка от этих пользователей - `64652$`.

 <a id="4"></a>
### 4. Дневные затраты на пользователя по источнику  [*(Оглавление)*](#contents)
Введём новую метрику эффективности рекламы - дневные затраты на рекламу на пользователя по источнику. Она рассчитывается для каждого отдельного дня и равна объёму денежных средств, которые были израсходованы на то, чтобы пользователь перешёл по рекламе на сайт из данного рекламного источника. Иными словами, метрика для каждого дня равна отношению рекламных затрат по данному источнику к количеству посетителей, перешедших на сайт с данного источника. 

Необходимо рассчитать введённую метрику для каждого дня. Будем считать, что целевое действие было произведено тогда, когда сесия началась. Например, если посетитель провёл на сайте несколько минут с 23:50 14 апреля по 0:20 15 апреля, то его нужно включать в рассчёт за 14 апреля

В случае, если пользователь несколько раз в день заходил на сайт, будем считать каждое его посещение отдельно (т.е. количество сессий в день, а не количество уникальных `uid`).

Назовем новую метрику `DACUS` - Daily Advertisment Cost per User per Source.

In [33]:
adv_source_per_user_costs = \
''' SELECT DATE(ac.dt::TIMESTAMP) AS date,                                           -- дата
           v.sourceid AS source_id,                                                  -- источник рекламы
           COUNT(v.uid) AS visits,                                                   -- количество посещений
           SUM(ac.costs::numeric)/COUNT(v.uid) AS costs,                             -- расходы на рекламу
           SUM(ac.costs::numeric)/(COUNT(v.uid)*COUNT(v.uid)) AS dacus               -- DACUS
    
    FROM advertisment_costs ac                                                       -- объединим таблицу расходов на рекламу
    INNER JOIN visits v ON DATE(ac.dt::TIMESTAMP) = DATE(v.startts::TIMESTAMP) AND   -- с таблицей посещений по общей дате 
                          ac.sourceid = v.sourceid                                   -- и номеру источника рекламы
    
    GROUP BY date, source_id                                                         -- сгруппируем по дате и источнику рекламы
    
    ORDER BY date, source_id;                                     -- отсортируем по возрастанию даты и номера источника рекламы
'''

In [34]:
adv_source_per_user_costs = pd.io.sql.read_sql(adv_source_per_user_costs, con = engine)

In [35]:
adv_source_per_user_costs

,date,source_id,visits,costs,dacus
0,2017-06-01,1,68,75.20,1.105882
1,2017-06-01,2,85,132.56,1.559529
2,2017-06-01,3,185,349.38,1.888541
3,2017-06-01,4,141,76.74,0.544255
4,2017-06-01,5,121,73.24,0.605289
...,...,...,...,...,...
2536,2018-05-31,3,404,822.30,2.035396
2537,2018-05-31,4,508,492.54,0.969567
2538,2018-05-31,5,253,398.04,1.573281
2539,2018-05-31,9,62,27.37,0.441452


---

Проверим в pandas.

In [36]:
# оставим в столбце с датой и временем только дату
visits['startts'] = visits['startts'].astype('datetime64[D]')
visits = visits.rename(columns = {"startts" : "date"})

In [37]:
# оставим в столбце с датой и временем только дату
adv_costs['dt'] = adv_costs['dt'].astype('datetime64[D]')
adv_costs = adv_costs.rename(columns = {"dt" : "date"})

In [38]:
# сгруппируем по дате и источнику рекламы
visits = visits.groupby(['date', 'sourceid'])['uid'].count().reset_index()
visits = visits.rename(columns = {"uid" : "visits"})

In [39]:
# соединим датасеты по условию равенства date и sourceid (inner merge)
dacus = pd.merge(adv_costs, visits, on=['date', 'sourceid'], how = 'inner')

In [40]:
# отбросим лишний столбец, отсортируем по возрастанию даты и номера рекламного источника
dacus = dacus.drop('id', axis = 1).sort_values(by = ['date','sourceid'])

In [41]:
# расчитаем метрику DACUS как отношение ежедневных затрат по каждому источнику 
# к количеству ежедневных посетителей от этого источника
dacus['dacus'] = (dacus['costs']/ dacus['visits'])

In [42]:
dacus.head()

,sourceid,date,costs,visits,dacus
0,1,2017-06-01,75.20,68,1.105882
363,2,2017-06-01,132.56,85,1.559529
726,3,2017-06-01,349.38,185,1.888541
1089,4,2017-06-01,76.74,141,0.544255
1452,5,2017-06-01,73.24,121,0.605289


In [43]:
len(dacus)

2541

In [44]:
dacus['dacus'].describe().round(2)

count    2541.00
mean        0.93
std         0.68
min         0.00
25%         0.52
50%         0.78
75%         1.16
max        12.10
Name: dacus, dtype: float64

Дневные затраты на рекламу на пользователя по источнику в среднем (медианное значение) составляют `0.78$`.